In [1]:
from googleapiclient.discovery import build

api_key = ""
youtube = build('youtube','v3',developerKey=api_key)

In [2]:
def get_channel_details(channel_id):
    try:
        ch_data = []
        response = youtube.channels().list(part = 'snippet,contentDetails,statistics',
                                        id= channel_id).execute()

        for i in range(len(response['items'])):
            data = dict(Channel_id = response['items'][i]['id'],
                        Channel_name = response['items'][i]['snippet']['title'],
                        Description = response['items'][i]['snippet']['description'],
                        Playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                        Subscribers = int(response['items'][i]['statistics']['subscriberCount']),
                        Views = int(response['items'][i]['statistics']['viewCount']),
                        Total_videos = int(response['items'][i]['statistics']['videoCount']),
                        Country = response['items'][i]['snippet'].get('country')
                        )
            ch_data.append(data)
        return ch_data
    except Exception as e:
        print(e)

In [3]:
print(get_channel_details('UCZT1WGS_kg2sg-kVpWZKEkg'))

[{'Channel_id': 'UCZT1WGS_kg2sg-kVpWZKEkg', 'Channel_name': 'Sha Boo Three', 'Description': 'Are you a fan of success stories? Do you look for opportunities to push your boundaries ? Sha Boo 3 will share the lifestyle of successful people from all around the world. From Cinema stars to Politicians, Athletes to Entrepreneurs, Scholars to Scientists etc \n\nKnow what made them successful! Subscribe now.\n\nFor Business Enquiry: Rjshapromotion@gmail.com\n', 'Playlist_id': 'UUZT1WGS_kg2sg-kVpWZKEkg', 'Subscribers': 1800000, 'Views': 222614384, 'Total_videos': 645, 'Country': 'IN'}]


In [4]:
playlistid = (get_channel_details('UCZT1WGS_kg2sg-kVpWZKEkg'))[0]['Playlist_id']
print(playlistid)

UUZT1WGS_kg2sg-kVpWZKEkg


In [5]:
def get_channel_videoids(channel_id):
    try:
        video_ids = []
        # get Uploads playlist id
        
        playlist_id = (get_channel_details(channel_id))[0]['Playlist_id']
        next_page_token = None
        
        while True:
            res = youtube.playlistItems().list(playlistId=playlist_id, 
                                            part='snippet', 
                                            maxResults=50,
                                            pageToken=next_page_token).execute()
            
            for i in range(len(res['items'])):
                video_ids.append(res['items'][i]['snippet']['resourceId']['videoId'])
            next_page_token = res.get('nextPageToken')
            
            if next_page_token is None:
                break
        return video_ids
    except Exception as e:
        print(e)

In [6]:
vedio_ids = get_channel_videoids('UUZT1WGS_kg2sg-kVpWZKEkg')
vedio_ids

'items'
'NoneType' object is not subscriptable


In [7]:
vedio_ids = get_channel_videoids('UCiEmtpFVJjpvdhsQ2QAhxVA')
vedio_ids

['Rwh7Na4ynyc',
 '8Q7_sSTcy4c',
 'rYt4RFSxYuc',
 'fEMROKW3pDE',
 'l1RWu7qU1sc',
 '5DoR_veNWOA',
 'j2mttQysedU',
 'rT6jXTptXFc',
 'pq0vFGq5uxU',
 'nKAvvlrxQJs',
 'a4Y_GtnhJHY',
 '_GeNIXgx_wo',
 'jZnhmNBTqvI',
 'SyNGmZbfVic',
 'rXcGhHiWQlE',
 'pzo46i2gEaw',
 'HghdMRFmLQ0',
 '3N2Jqhe4cx8',
 '6ApXNp2xjfU',
 'LWcdQnYkBYk',
 'H07EMEHuWpA',
 'tTElUHfNxkw',
 'r_6oj5WJE-4',
 'qXmyFh8wAVE',
 'cnt2ZKJUEIc',
 '6oSbkD9EAG0',
 'iUAee_AC0cg',
 '3AkjpwQbkB0',
 'MwW5bF_m5tk',
 'JWWSyvawVxc',
 'pBJ1PnRdX7M',
 'zUQc_p3y2fU',
 '5QZWMRuK9CQ',
 '5nAb0PNEMa8',
 'V2Dza5ePKkY',
 'JkecZD3-yvg',
 'ygBvQ0C8LDE',
 '5seveAAnIrg',
 'dcipC3XXE3I',
 '-rrSNQLBndg',
 'fD25mi939Zo',
 'd_NjlbuUtGE',
 'Jbvo_rZ-yYg',
 'JWQQ5VgsJ2E',
 'od0VBQQ39f4',
 'MGgmwHXHCNI',
 'MZa5R4PrTqo',
 '4MPneA3QHsE',
 'TMhrSf6RjhY',
 'Q43Ai1NyVG4',
 'cWDZW6gULLk',
 'j2zpf4VShZg',
 'AxdfDBxM3nU',
 '10BJXHVPG0k',
 'AsfPRIxatOo',
 'VogJLaMGOVM',
 'uBvPDlWFQKY',
 'lbsi5jnAixw',
 '9cwd1FSGpl8',
 '94sAz-_FkQY',
 'gNZQ-JYc18E',
 '2MZJnA12m-Q',
 'fkhJMY

In [8]:
import re

In [9]:
def convert_duration(duration):
    regex = r'PT(\d+H)?(\d+M)?(\d+S)?'
    match = re.match(regex, duration)
    if not match:
        return '00:00:00'
    hours, minutes, seconds = match.groups()
    hours = int(hours[:-1]) if hours else 0
    minutes = int(minutes[:-1]) if minutes else 0
    seconds = int(seconds[:-1]) if seconds else 0
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return '{:02d}:{:02d}:{:02d}'.format(int(total_seconds / 3600), int((total_seconds % 3600) / 60), int(total_seconds % 60))

In [10]:
def get_video_details(v_ids):
    try:
        videos = []
    
        for vid in v_ids:
            response = youtube.videos().list(
                        part="snippet,contentDetails,statistics",
                        id=vid).execute()
            for video in response['items']:
                video_details = dict(Channel_name = video['snippet']['channelTitle'],
                                    Channel_id = video['snippet']['channelId'],
                                    Video_id = video['id'],
                                    Title = video['snippet']['title'],
                                    Tags = ','.join(video['snippet'].get('tags',["NA"])),
                                    Thumbnail = video['snippet']['thumbnails']['default']['url'],
                                    Description = video['snippet']['description'],
                                    Published_date = video['snippet']['publishedAt'],
                                    Duration = convert_duration(video['contentDetails']['duration']),
                                    Views = int(video['statistics']['viewCount']),
                                    Likes = int(video['statistics'].get('likeCount',0)),
                                    Dis_Likes = int(video['statistics'].get('dislikeCount',0)),
                                    Comments = int(video['statistics'].get('commentCount',0)),
                                    Favorite_count = int(video['statistics']['favoriteCount']),
                                    Definition = video['contentDetails']['definition'],
                                    Caption_status = video['contentDetails']['caption']
                                )
                videos.append(video_details)
        return videos
    except Exception as e:
        print(e)

In [11]:
for i in get_video_details(['qXmyFh8wAVE','iUAee_AC0cg']):
    print(i['Duration'])

00:02:13
00:01:12


In [12]:
def get_comments_details(v_id):
    try:
        comment_data = []
        try:
            response = youtube.commentThreads().list(part="snippet,replies",
                                                    videoId=v_id).execute()

            for cmt in response['items']:
                data = dict(Comment_id = cmt['id'],
                            Video_id = cmt['snippet']['videoId'],
                            Comment_text = cmt['snippet']['topLevelComment']['snippet']['textDisplay'],
                            Comment_author = cmt['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                            Comment_posted_date = cmt['snippet']['topLevelComment']['snippet']['publishedAt'],
                            Like_count = int(cmt['snippet']['topLevelComment']['snippet']['likeCount']),
                            Reply_count = int(cmt['snippet']['totalReplyCount'])
                            )
                comment_data.append(data)
        except:
            pass
        
        return comment_data
    except Exception as e:
        print(e)

In [13]:
vi="cWDZW6gULLk"
try:
    resp=youtube.commentThreads().list(part="snippet,replies",videoId=vi).execute()
except:
    resp=[]
if "items" not in resp:
    print("yes")

yes


In [15]:
for x in vedio_ids:
    print(x)
    for i in get_comments_details(x):
        print(i)

Rwh7Na4ynyc
8Q7_sSTcy4c
rYt4RFSxYuc
fEMROKW3pDE
{'Comment_id': 'UgymssDkZJRkaQxZ1Qh4AaABAg', 'Video_id': 'fEMROKW3pDE', 'Comment_text': 'Give me those guys contact details. I&#39;m also Non iT working prop', 'Comment_author': '@PJS718', 'Comment_posted_date': '2024-02-25T15:36:24Z', 'Like_count': 0, 'Reply_count': 1}
l1RWu7qU1sc
5DoR_veNWOA
{'Comment_id': 'Ugx-amn_b4h0_1bJZZp4AaABAg', 'Video_id': '5DoR_veNWOA', 'Comment_text': 'Join pana contact number sent me sir', 'Comment_author': '@sdartlifestyle2714', 'Comment_posted_date': '2024-02-22T15:16:56Z', 'Like_count': 0, 'Reply_count': 1}
j2mttQysedU
rT6jXTptXFc
pq0vFGq5uxU
nKAvvlrxQJs
a4Y_GtnhJHY
_GeNIXgx_wo
jZnhmNBTqvI
SyNGmZbfVic
rXcGhHiWQlE
pzo46i2gEaw
HghdMRFmLQ0
3N2Jqhe4cx8
6ApXNp2xjfU
LWcdQnYkBYk
H07EMEHuWpA
tTElUHfNxkw
r_6oj5WJE-4
qXmyFh8wAVE
cnt2ZKJUEIc
6oSbkD9EAG0
iUAee_AC0cg
3AkjpwQbkB0
MwW5bF_m5tk
JWWSyvawVxc
pBJ1PnRdX7M
zUQc_p3y2fU
5QZWMRuK9CQ
5nAb0PNEMa8
{'Comment_id': 'UgwaSQDRuYnX118h59l4AaABAg', 'Video_id': '5nAb0PNEMa8'